In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from tifffile import imsave
from skimage.filters import threshold_otsu

In [7]:
import skimage
skimage.__version__

'0.18.2'

In [2]:
path_stacks_dir ='./data_stacks'
path_frames_dir = './data_frames'

info_file_path = './otsu_selPlanes.csv'

## Find the middle plane in each stack and save separately

In [3]:
set_list = os.listdir(path_stacks_dir)

In [6]:
im_data_list=[]

for data_set in set_list:

    set_path = os.path.join(path_stacks_dir,data_set)

    exp_list = os.listdir(set_path)

    for data_exp in exp_list:

        exp_path = os.path.join(set_path,data_exp)

        file_list = [x for x in os.listdir(exp_path) if x[-3:]=='ims']

        i = 0
        for im_file in file_list:
            
            print(im_file)
            
            # open imaging file
            file_path = os.path.join(exp_path,im_file)
            f = h5py.File(file_path,'r')

            # extract channel data
            ch_00 = np.array(f['DataSet']['ResolutionLevel 0']['TimePoint 0']['Channel 0']['Data'])
            ch_01 = np.array(f['DataSet']['ResolutionLevel 0']['TimePoint 0']['Channel 1']['Data'])

            f.close()

            # find middle plane
            segm = ch_01 > threshold_otsu(ch_01)
            selected_frame = np.argmax(np.sum(ch_01*segm,axis=(1,2)))

            # choose frames
            ch_00_frame = ch_00[selected_frame,::1,::1]
            ch_01_frame = ch_01[selected_frame,::1,::1]

            # save frames
            imsave(os.path.join(path_frames_dir,f'{data_set}_{data_exp}_{str(i).zfill(2)}_ch00.tiff'),ch_00_frame)
            imsave(os.path.join(path_frames_dir,f'{data_set}_{data_exp}_{str(i).zfill(2)}_ch01.tiff'),ch_01_frame)

            # create data frame
            im_data = pd.DataFrame([selected_frame],columns=['selected_plane'])

            # add info of these measurements
            im_data['im_stack'] = im_file
            im_data['im_frame'] = f'{data_set}_{data_exp}_{str(i).zfill(2)}'
            im_data['condition'] = data_exp
            im_data['set'] = data_set
            im_data['total_z'] = ch_01.shape[0]

            im_data_list.append(im_data)

            i = i + 1

        

# put all together
im_data_all = pd.concat(im_data_list,ignore_index=True)
im_data_all.to_csv(info_file_path,index=False)

PA treated - 10min_HG003-PA-10m-NR and VAN - FOV01.ims
PA treated - 10min_HG003-PA-10m-NR and VAN - FOV02.ims
PA treated - 10min_HG003-PA-10m-NR and VAN - FOV03.ims
PA treated - 30min_HG003-PA alone-30m-NR and VAN - FOV01.ims
PA treated - 30min_HG003-PA alone-30m-NR and VAN - FOV02.ims
PA + VAN20 treated - 10min_HG003-PA plus VAN20-10m-NR and VAN - FOV01.ims
PA + VAN20 treated - 10min_HG003-PA plus VAN20-10m-NR and VAN - FOV02.ims
PA + VAN20 treated - 30min_HG003-PA plus VAN20-30m-NR and VAN - FOV01.ims
PA + VAN20 treated - 30min_HG003-PA plus VAN20-30m-NR and VAN - FOV02.ims
PA + VAN20 treated - 30min_HG003-PA plus VAN20-30m-NR and VAN - FOV03.ims
PA + VAN20 treated - 30min_HG003-PA plus VAN20-30m-NR and VAN - FOV04.ims
Untreated 30min_HG003-Untreated-30m-NR and VAN - FOV01.ims
Untreated 30min_HG003-Untreated-30m-NR and VAN - FOV02.ims
Untreated 30min_HG003-Untreated-30m-NR and VAN - FOV03.ims
VAN20 treated - 10min_HG003-VAN20-10m-NR and VAN - FOV01.ims
VAN20 treated - 10min_HG003-VAN